# MNIST with CNN

In [23]:
import tensorflow as tf


In [24]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


###  helper functions

In [25]:
# initializing weights with truncated normal distribution and returning them as variable
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)    

In [26]:
# initializing biases with a constant and returning as a variable
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Creating a 2D convolution wrapper using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter / kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op
performs the following:

1. Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,
   filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.


In [27]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W, strides = [1, 1, 1, 1], padding= "SAME")

Creating a max pooling layer, again using built in TF functions:

Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch, height, width, channels]` and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. 

In [28]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1],
                          strides = [1, 2, 2, 1],
                          padding = 'SAME')

creating a convolutional layer using conv2d function with ReLu activation

In [29]:
def convolutional_layer(input_x, kernel_shape):
    W = init_weights(kernel_shape)
    b = init_bias([kernel_shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

creating a dense layer with relu activation

In [30]:
# number of neurons in this layer  = op_size
def dense_layer(input_layer, op_size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, op_size])
    b = init_bias([op_size])
    print(W.shape,b.shape)
    return tf.matmul(input_layer,W) + b

##  placehlders

In [31]:
x = tf.placeholder(tf.float32, shape = [None, 784])
y_true = tf.placeholder(tf.float32, shape = [None, 10])
rate1 = tf.placeholder(tf.float32) # for Dropout layer

## Layers

In [32]:
# input layer
x_image = tf.reshape(x, [-1,28,28,1])


# convo_1, using 5x5x1 filter, and 32 such filters
convo_1 = convolutional_layer(x_image, kernel_shape = [6, 6, 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)
# size of output of this layer is [-1,14,14,32]


# convo_2, using 5x5x32 filter, and 64 such filters 
convo_2 = convolutional_layer(convo_1_pooling, kernel_shape = [6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)
# size of output of this layer is [-1, 7, 7, 64]

# flattening and applying a fully connected layer with 1024 neurons
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_1 = tf.nn.relu(dense_layer(convo_2_flat, 1024))
# size of output of this layer is [-1,1024]


# Including Dropouts
full_layer_1_dropout = tf.nn.dropout(full_layer_1, rate = 0.5)

# dense layer2 with 10 neurons
y_pred = dense_layer(full_layer_1_dropout, 10)


(3136, 1024) (1024,)
(1024, 10) (10,)


## loss function

In [33]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true , logits = y_pred))

## optimizer

In [34]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

##  initialize variables

In [35]:
init = tf.global_variables_initializer()

## session

In [38]:
steps = 500
accuracy1 = []

with tf.Session() as sess:
    sess.run(init)
    
    
    for i in range(steps):
        x_batch, y_batch = mnist.train.next_batch(100)
        feed = {x: x_batch,
               y_true: y_batch, 
               rate1 : 0.5}
        sess.run(train, feed_dict= feed)
        
        
        if i%50 == 0:
            print('Current step = {}'.format(i))
            
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,  tf.float32))
            accuracy = sess.run(acc, feed_dict = {x: mnist.test.images,
                                             y_true: mnist.test.labels,
                                             rate1: 1.0})
            print('Accuacy ::',accuracy)
            accuracy1.append(accuracy)
            
            print('\n')
    
    

Current step = 0
Accuacy :: 0.0929


Current step = 50
Accuacy :: 0.5414


Current step = 100
Accuacy :: 0.738


Current step = 150
Accuacy :: 0.8008


Current step = 200
Accuacy :: 0.8407


Current step = 250
Accuacy :: 0.8635


Current step = 300
Accuacy :: 0.8845


Current step = 350
Accuacy :: 0.896


Current step = 400
Accuacy :: 0.9057


Current step = 450
Accuacy :: 0.9145


